# The First Example
Usually you start off with a "hello world" example.  But "hello world" would be too boring in F# so let us start with something a little more interesting and fun.  In this first example we will create a dataset (i.e. a list) that we will show in a histogram.  The example will show you pretty much all you need to get started working with data in F#.  It is ok if you do not understand much of what is going on.  In later chapters you will learn more about how this example works.

In [2]:
type PersonPolicy = 
    {
        PersonId: string;
        PolicyNumber: string;
        Premium: decimal;
    }
    
let data = 
    [ 
        { PersonId = "P1"; PolicyNumber = "Pol01"; Premium = 100m };
        { PersonId = "P1"; PolicyNumber = "Pol02"; Premium = 200m };
        { PersonId = "P2"; PolicyNumber = "Pol03"; Premium = 150m };
        { PersonId = "P3"; PolicyNumber = "Pol04"; Premium = 250m };
        { PersonId = "P3"; PolicyNumber = "Pol05"; Premium = 350m };
    ]
    
data

index,PersonId,PolicyNumber,Premium
0,P1,Pol01,100
1,P1,Pol02,200
2,P2,Pol03,150
3,P3,Pol04,250
4,P3,Pol05,350


In line 1 you define a `type`.  A type defines the shape of your data.  The `PersonPolicy` type defines how each row or item in your dataset will look like.  Each row has a person ID, a policy number and a premium.  This is similar to how you would define columns in an Excel spreadshet or in a database table. In a later chaper "Structuring Data", you will learn more about types.

The value `data` in line 8 is a list of persons where each person can have one or more policies.  This is your dataset.  The output in line 17 shows the `data` value as a nice table.

Let us draw a diagram.  We want a bar diagram with the policy number on one axis and the premium on the other.  That means we want to create data for the two axis from our existing `data` values.  F# has a built-in function in the so-called `List` module for creating data or "mapping" data called `map`.  So we will use `List.map` to create data for the diagram.

In [3]:
let getPolicyNumberAndPremium row =
    row.PolicyNumber, row.Premium
    
let graphData = data |> List.map getPolicyNumberAndPremium

graphData

index,Item1,Item2
0,Pol01,100
1,Pol02,200
2,Pol03,150
3,Pol04,250
4,Pol05,350


There are a couple of things going on here.

In line 1 you define a function called `getPolicyNumberAndPremium` that takes one argument with the name `row`.  F# will figure out on its own that `row` is a `PersonPolicy`.  We say that "row is of type PersonPolicy".

Next in line 4, we pass our dataset `data` along with the function `getPolicyNumberAndPremium` into the `List.map` function.  `List.map` will call `getPolicyNumberAndPremium` and return a new list with rows containing the pair `(policynumber, premium)`.  This list is shown in the table.

Note also the use of the so called "pipe" operator `|>`.  The pipe operator "pipes" data into a function and it is extremely important and useful when working with data in F#.  We will get back to it later in a later chapter.

You will learn more about functions and how functions can be passed to other functions as arguments in the chapter "Functions".

We are now ready to create the plot.

In [23]:
graphData
|> Chart.Bar
|> Chart.WithLabels ["Policies"; "Premiums"]

That looks nice but what we really want is to group the policies and show the total premium for each person.

Let us create a type that holds the premium per person.

In [24]:
type PersonPremium = 
    {
        PersonId: string;
        Premium: decimal;
    }

We also need a function that sums the premiums for a list of `PersonPolicy`.  We will use the function `List.sumBy` to do that.

In [25]:
let sumPremium (personPolicies: PersonPolicy list) =
    personPolicies |> List.sumBy (fun personPolicy -> personPolicy.Premium)

Wait, what happened in line 2 above?  What is the `fun personPolicy -> personPolicy.Premium` part all about?  This is actually a function without a name.  So you can create a function "on-the-fly" with the `fun` keyword, and pass that function to another function, `List.sumBy`.

We can test it by summing the premiums for our entire dataset.

In [26]:
sumPremium data

1050

That looks just about right, except we do not want to get the total premium.  Instead we want the premium for each person so first we need to group the dataset per person and then sum the premiums.  Fortunately, F# has a built-in function for grouping lists: `List.groupBy`.  After grouping we use `List.map` to transform the grouped data into an list of `PersonPremium` elements and we apply our function `sumPremium` to sum the premiums for each person.

In [27]:
let groupedByPerson = 
    data
    |> List.groupBy (fun personPolicy -> personPolicy.PersonId)
    |> List.map (fun (personId, personPolicies) -> 
        {
            PersonId = personId; 
            Premium = sumPremium personPolicies 
        })
    
groupedByPerson


index,PersonId,Premium
0,P1,300
1,P2,150
2,P3,600


Now it is starting to get a bit more complicated.  In line 3 you use `List.groupBy` to group your data by `PersonId` by passing 
`fun personPolicy -> personPolicy.PersonId`.  After grouping, you have a new dataset with a list of policies for each personId, which you can see in the `fun (personId, personPolicies)` part of line 4.  In line 5 and forward we create new rows containing `PersonPremium` elements, with the premiums summed for each person which is done in line 7.

That is pretty neat! Let's plot it.

In [28]:
groupedByPerson
|> List.map (fun g -> g.PersonId, g.Premium)
|> Chart.Bar
|> Chart.WithLabels ["Person"; "Premiums"]

## Conclusion
This example hopefully gave you an idea of the power of F#.  It showed you how you can create datasets as lists and how you can transform and plot data.  Don't worry if you did not understand all of it.  I would be surprised if you did.  But hopefully you can see from the example that F# gives you a consistent and compact way of transforming and aggregating data.  We will dive deeper into the concepts of data and functions in the rest of the guide.
